In [1]:
##### CASO PRÁCTICO DE BUSINESS INTELLIGENT ######

# OBJETIVO: encontrar la ubicación más adecuada, en todo el mundo, para la siguiente empresa:

    # Actividad: videojuego
    
    # Compañía de 50 empleados.
        # 20 desarrolladores
        # 20 diseñadores creativos
        # 10 ejecutivos 

    # Condiciones: 
        # Entorno con buen ratio entre grandes compañías y startups.
        # Entorno que cubra intereses de eqipo: 
            # empresas de diseño
            # empresas de desarrollo
            # consultoras y empresas para hacer negocio
        # Evitar compañías muy viejas
        
    # Estrategia
        # 1ª Realizamos un filtrado desde la base de datosque elimine:
            # Empresas sin información sobre ubicación
            # Empresas creadas anteriormente a 1996.
        
        # 2ª Asignamos un rating a cada empresa en función de:
            # Su número de empleados.
            # Su valor.
            # Su antigüedad.
            # Su actividad.
        
        # 3ª Seleccionamos la compañía con mejor ubicación en función del rating
        # de las empresas de su entorno (radio de 10 km).



In [2]:
# Importación de las librerías necesarias

import pymongo

import pandas as pd


In [3]:
# Realizamos la llamada a Mongoclient

MongoClient = pymongo.MongoClient

# Asignamos MongoClient() a la variable client 

client = MongoClient()

# La variable db se encargará de hacer la llamada a la base de datos companies

db = client.companies

In [4]:
#########                                                                       #######
#########   FUNCIONES PARA TRANSFORMACION DEL DOCUMENTO DE CADA COMPAÑÍA        #######
#########                                                                       #######


# Creación de toGeoJSON:
    # Input: lista con dos valores: la longitud y la latitud de la ubicación de la compañía.
    # Output: diccionario con ambas coordenadas en una sola lista y el establecimiento de "type".
    
def toGeoJSON(lista):
    return {
            "type": "Point",
            "coordinates":[lista[1],lista[0]]
        }

# Creación de toGeoJSON:
    # Input: lista co los valores de las rondas de inversión.
    # Output: entero con el total del valor de la inversión.

def getValuatio(funding_round):
    valor= 0
    for lista in funding_round:
        valor = lista['raised_amount']
    return valor

    

# Creación de company_Document() 
    
    # Input: documento (uno por compañía) tal y como se encuentra en la base de datos original.
    
    # Output: lista_dict: lista con diccionarios con la siguiente información sobre cadaa compañia:
        
        # Name (nombre), category_code (actividad),  number_of_employees (total de empleados),
        # valuation (valor de la compañía: así interpretamos la inversión obtenida en la ronda 
        # de inversiones), city(ciudad y códiigo del país de ubicación) y position (coordenadas). 
        
        # Para conseguir el valor de position usamos las función toGeoJso() anteriormente definida.
        
        # Para conseguir el valor de valuation usamos las función getValuatio() anteriormente definida.
        
def companyDocument(company):
    lista_dict = [
                {
                "name": company["name"],
                "category_code": company["category_code"],
                "number_of_employees": company["number_of_employees"],
                "founded_year": company["founded_year"],
                "valuation": getValuatio(company['funding_rounds']),
                "city": (key["city"],  key["country_code"]),  
                "position": toGeoJSON([key['latitude'],key['longitude']])   
                }
    for index in range(len(company['offices'])) for key in company['offices'] \
    if key['latitude'] != None and key['longitude'] != None
                ]
                
    return lista_dict
    


In [5]:
# A la variable filtered_comp, asignamos el filtrado de documentos de empresas que nos interesan:
    
    # que contengan las coordenadas: tanto latitud como longitud.
    
    # que hayan sido creadas con posterioridad a 1995,(las anteriores las condiramos antiguas)
    
    # que no hayan desaparecido.

# Por último nos aseguramos de extraer el id de cada documento.
    
filtered_companies = db.companies.find(
                    {"$and": [ 
                        { "offices.latitude": {"$exists": True} },
                        { "offices.longitude": {"$exists": True} }, 
                        { "founded_year": {"$gt": 1995} }, 
                        {"deadpooled_year": None}  
                            ]},
                    {"_id":0})


In [6]:
# Damos la forma y el contenido que nos interesa a los documentos que almacena filtered_companies:
    
        # Aplicamos companyDocument() (definida arriba) a cada documento.
        
        # y almacenamos cada documento (diccionario) en una lista, companies_docs 

companies_docs = list(map(lambda r: companyDocument(r),filtered_companies))

# Agrupamos todos los diccionarios de companies_docs en una sola lista.

geoCompanies = [element for lista in companies_docs for element in lista]


In [7]:
# Creamos un dataframe con los valores de geoCompanies.

df = pd.DataFrame(geoCompanies)

df.head()
  

,category_code,city,founded_year,name,number_of_employees,position,valuation
0,news,"(San Francisco, USA)",2004,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,...",5000000.0
1,web,"(San Carlos, USA)",1999,Postini,NaN,"{'type': 'Point', 'coordinates': [-122.247573,...",0.0
2,web,"(West Hollywood, USA)",2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",5000000.0
3,web,"(Menlo Park, USA)",2003,Gizmoz,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",5300000.0
4,web,"(San Francisco, USA)",2002,StumbleUpon,NaN,"{'type': 'Point', 'coordinates': [-122.419204,...",NaN


In [8]:
#########                                                 #######
#########   FUNCIONES PARA RATING DE CADA COMPAÑÍA        #######
#########                                                 #######

# Creamos rating() para calcular el rating de cada compañía: 

        #Input: df

        #Output: lista con el rating total de cada compañía.

    # Dentro de la rating() creamos cuatro funciones para asignar una putuación a cada empresa por su:
            # Su número de empleados.
            # Su valor.
            # Su antigüedad.
            # Su actividad.
      
            
def rating(df):            
# Creamos getRatingCode() para asignar un rating a cada compañía en función de su actividad
    
    #Input: código de actividad de la compañía
    
    #Output: puntuación por actividad.

    def getRatingCode(code):
        ratingCode = 0
        ratingCode = 5 if code == ("web" or "software" or "hardware" or "design" or "consulting") \
        else ratingCode
        ratingCode = 4 if code == ("advertising" or "games_video" or  "ecommerce") else ratingCode
        ratingCode = 3 if ("network_hosting" or "search" or "cleantech" or "analytics" or "messaging") \
        else ratingCode
        ratingCode = 2 if ("mobile" or "public_relations"  or "finance" or "fashion") else ratingCode
        ratingCode = 1 if ("biotec" or "security" or "semiconductor" or "education" or "medical" 
                           or "health" or "manufacturing" or "nanotech" or "social" or "music" 
                           or "news" or "travel" or "photo_video") else ratingCode
        return ratingCode


    # Creamos getRatingFunds() para asignar un rating a cada compañía en función de su valor:

        #Input: valor de la compañía

        #Output: puntuación por valor.


    def getRatingFunds(funds):
        ratingFunds = 0
        ratingFunds = 5 if funds >= 100000000 else ratingFunds
        ratingFunds = 4 if funds >= 8000000 and funds < 100000000 else ratingFunds
        ratingFunds = 3 if funds >= 3000000 and funds < 8000000 else ratingFunds
        ratingFunds = 2 if funds >= 1500000 and funds < 3000000 else ratingFunds
        ratingFunds = 1 if funds >= 500000 and funds < 1500000 else ratingFunds
        return ratingFunds



    # Creamos getRatingEmpl() para asignar un rating a cada compañía en función de su número de empleados:

        #Input: número de empleados de la compañía

        #Output: puntuación por número de empleados.

    def getRatingEmpl(employees):
        ratingEmpl = 0
        ratingEmpl = 5 if employees >= 50000 else ratingEmpl
        ratingEmpl = 4 if employees >= 10000 and employees < 50000 else ratingEmpl
        ratingEmpl = 3 if employees >= 1000 and employees < 10000 else ratingEmpl    
        ratingEmpl = 2 if employees >= 200 and employees < 1000 else ratingEmpl
        ratingEmpl = 1 if employees >= 50 and employees < 200 else ratingEmpl
        return ratingEmpl

    # Creamos getRatingYear() para asignar puntos a la compañía si se trata de una start up:

        #Input: año de fundación de la compañía

        #Output: puntuación por año de la fundación.

    def getRatingYear(year):        
        ratingYear = 0
        ratingYear = 5 if year > 2010 else ratingYear
        return ratingYear    

    return df.number_of_employees.map(getRatingEmpl) + df.valuation.map(getRatingFunds) \
    + df.category_code.map(getRatingCode) + df.founded_year.map(getRatingYear)


In [9]:
# Ejecutamos rating() sobre el df y asignamos los valores obtenidos a rating.

rating = rating(df)


In [10]:
# Creamos una nueva columna 'rating' en df con la puntuación obtenida por cada empresa.

df['rating'] = rating

df.head()

,category_code,city,founded_year,name,number_of_employees,position,valuation,rating
0,news,"(San Francisco, USA)",2004,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,...",5000000.0,5
1,web,"(San Carlos, USA)",1999,Postini,NaN,"{'type': 'Point', 'coordinates': [-122.247573,...",0.0,1
2,web,"(West Hollywood, USA)",2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",5000000.0,4
3,web,"(Menlo Park, USA)",2003,Gizmoz,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",5300000.0,4
4,web,"(San Francisco, USA)",2002,StumbleUpon,NaN,"{'type': 'Point', 'coordinates': [-122.419204,...",NaN,1


In [11]:
# A partir de df generamos un nuevo data son con el que crearemos las colección 'location'

df.to_json("comp_with_coords.json", orient="records", lines=True)

In [12]:
# En companies almacenamos los diccionarios de cada compañía con su rating incorporado.

companies = list(db.location.find({}))

# Generamos un dataframe con los valores de companies

dfRating = pd.DataFrame(companies)
dfRating.head()

,_id,category_code,city,founded_year,name,number_of_employees,position,rating,valuation
0,5c6bc9be0fd4a80911c20ec7,news,"[San Francisco, USA]",2004,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,...",5,5000000.0
1,5c6bc9be0fd4a80911c20ec8,web,"[San Carlos, USA]",1999,Postini,NaN,"{'type': 'Point', 'coordinates': [-122.247573,...",1,0.0
2,5c6bc9be0fd4a80911c20ec9,web,"[West Hollywood, USA]",2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",4,5000000.0
3,5c6bc9be0fd4a80911c20eca,web,"[Menlo Park, USA]",2003,Gizmoz,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",4,5300000.0
4,5c6bc9be0fd4a80911c20ecb,web,"[San Francisco, USA]",2002,StumbleUpon,NaN,"{'type': 'Point', 'coordinates': [-122.419204,...",1,NaN


In [13]:
# Definimos getAreasRat() para hallar el rating que suman las empresas en un radio
# de 10 km alrededor de una compañía. La empresa con mayor rating en su área, 
# representará las coordenadas elegidas para la ubicación de nuestra empresa.

    # Input: lista de compañías
    
    # Output: lista con el valor de rating del área de cada compañía.

def getAreasRat(companylist):
    areasRating = [sum(i['rating'] for i in list(
        db.location.find({
              "position": {
                "$near": {
                  "$geometry": {
                    "type": "Point",
                    "coordinates": companylist[i]['position']['coordinates']
                  },
                  "$minDistance": 0,
                  "$maxDistance":10000
                }
              }
            })
            ))   for i in range(len(companylist))]
    return areasRating
  

In [14]:
# Ejecutamos getAreasRat() y almacenamos los valores en listOfRating

listOfRating = getAreasRat(companies)


In [15]:
# Añadimos listofRating al df para añadir el valor del área de cada compañía

dfRating['area_rating'] = listOfRating
dfRating.head()


,_id,category_code,city,founded_year,name,number_of_employees,position,rating,valuation,area_rating
0,5c6bc9be0fd4a80911c20ec7,news,"[San Francisco, USA]",2004,Digg,60.0,"{'type': 'Point', 'coordinates': [-122.394523,...",5,5000000.0,2730
1,5c6bc9be0fd4a80911c20ec8,web,"[San Carlos, USA]",1999,Postini,NaN,"{'type': 'Point', 'coordinates': [-122.247573,...",1,0.0,1149
2,5c6bc9be0fd4a80911c20ec9,web,"[West Hollywood, USA]",2006,Geni,18.0,"{'type': 'Point', 'coordinates': [-118.393064,...",4,5000000.0,384
3,5c6bc9be0fd4a80911c20eca,web,"[Menlo Park, USA]",2003,Gizmoz,NaN,"{'type': 'Point', 'coordinates': [-122.169472,...",4,5300000.0,1151
4,5c6bc9be0fd4a80911c20ecb,web,"[San Francisco, USA]",2002,StumbleUpon,NaN,"{'type': 'Point', 'coordinates': [-122.419204,...",1,NaN,2730


In [16]:
# Preparamos el df para la posterior lectura de las coordenadas en Tableau.

# Primero generamos un df con una columna para longitud y otro para latitud.

coords_df = pd.DataFrame(pd.DataFrame(dfRating["position"].values.flatten().tolist())["coordinates"]
            .values.tolist())

# Unimos dfRating con el df de las coordeenadas.
clean_df = pd.concat([dfRating,coords_df],axis=1)

# Eliminamos las columanas que no son útiles.
clean_df.drop(['_id',"position"], inplace=True, axis=1)

# Renombramos las columnas de coordenadas.
clean_df = clean_df.rename({0:"long",1:"lat"}, axis=1)

clean_df.head()

,category_code,city,founded_year,name,number_of_employees,rating,valuation,area_rating,long,lat
0,news,"[San Francisco, USA]",2004,Digg,60.0,5,5000000.0,2730,-122.394523,37.764726
1,web,"[San Carlos, USA]",1999,Postini,NaN,1,0.0,1149,-122.247573,37.506885
2,web,"[West Hollywood, USA]",2006,Geni,18.0,4,5000000.0,384,-118.393064,34.090368
3,web,"[Menlo Park, USA]",2003,Gizmoz,NaN,4,5300000.0,1151,-122.169472,37.484130
4,web,"[San Francisco, USA]",2002,StumbleUpon,NaN,1,NaN,2730,-122.419204,37.775196


In [27]:
# Localizamos la empresa con mejor ubicación

selected_comp = clean_df.loc[clean_df['area_rating'].idxmax()]
selected_name = selected_comp['name']
selected_city = selected_comp['city']
selected_coord = [selected_comp['long'],selected_comp['lat']]
display(selected_name)
display(selected_city)
display(selected_coord)

'Etsy'

['Brooklyn', 'USA']

[-73.983268, 40.694731]

In [18]:
# Generamos un dataframe definitivo con las datos seleccionados.
clean_df.to_json('visualize_location.json', orient="records")

In [19]:
selectedArea = db.location.find({
          "position": {
            "$near": {
              "$geometry": {
                "type": "Point",
                "coordinates": selected_coord
              },
              "$minDistance": 0,
              "$maxDistance":10000
            }
          }
        })


In [20]:
dfSelected = pd.DataFrame(list(selectedArea))

In [21]:
coords_df_selec = pd.DataFrame(pd.DataFrame(dfSelected["position"].values.flatten().tolist())["coordinates"].values.tolist())
clean_df_selec = pd.concat([dfSelected,coords_df_selec],axis=1)
clean_df_selec.drop(['_id',"position"], inplace=True, axis=1)
clean_df_selec = clean_df_selec.rename({0:"long",1:"lat"}, axis=1)
display(clean_df_selec.head())
display(clean_df_selec.describe())

,category_code,city,founded_year,name,number_of_employees,rating,valuation,long,lat
0,ecommerce,"[Brooklyn, USA]",2005,Etsy,300.0,7,40000000.0,-73.983268,40.694731
1,ecommerce,"[Brooklyn, USA]",2006,Pond5,25.0,2,500000.0,-73.987853,40.703288
2,ecommerce,"[Brooklyn, USA]",2006,Pond5,25.0,2,500000.0,-73.987853,40.703288
3,ecommerce,"[Brooklyn, USA]",2006,Pond5,25.0,2,500000.0,-73.987853,40.703288
4,ecommerce,"[Brooklyn, USA]",2006,Pond5,25.0,2,500000.0,-73.987853,40.703288


,founded_year,number_of_employees,rating,valuation,long,lat
count,868.000000,603.000000,868.000000,8.110000e+02,868.000000,868.000000
mean,2004.062212,3408.538972,3.412442,5.590414e+07,-73.990680,40.740036
std,3.877989,9173.284912,2.726762,1.704284e+08,0.017341,0.020641
min,1996.000000,0.000000,1.000000,0.000000e+00,-74.081365,40.621591
25%,2000.000000,15.000000,1.000000,0.000000e+00,-74.001169,40.730509
50%,2005.000000,50.000000,2.000000,5.000000e+05,-73.990411,40.743616
75%,2007.000000,160.000000,5.000000,1.000000e+07,-73.983649,40.753631
max,2012.000000,50000.000000,10.000000,1.500000e+09,-73.886452,40.781159


In [22]:
clean_df_selec.to_json('visualize_selected_location.json', orient="records")

![title](img/etsy_position.png)